# RiskDAG Quickstart - Complex DAG Example

This notebook demonstrates RiskDAG with a more complex pipeline matching Airflow's visualization style.

## 1. Import and Setup

In [8]:
from riskdag import RiskDAG, RiskNode, LatentRiskNode
from riskdag.visualization import visualize_risk_dag, create_interactive_dashboard

from scipy import stats

print("RiskDAG imported successfully!")

RiskDAG imported successfully!


## 2. Create Complex Pipeline

Like Airflow's example: A >> B >> C >> D and B >> E >> D

In [9]:
with RiskDAG('complex_etl', time_scale='hour') as dag:
    # Latent infrastructure risks
    cloud = LatentRiskNode(
        'cloud',
        p_fail=0.01,  # 1% daily
        loss_dist=stats.norm(loc=5000, scale=1000),
        user_time_scale='day'
    )
    
    database = LatentRiskNode(
        'database',
        p_fail=0.02,  # 2% daily
        loss_dist=stats.norm(loc=10000, scale=2000),
        user_time_scale='day'
    )
    
    network = LatentRiskNode(
        'network',
        p_fail=0.015,  # 1.5% daily
        loss_dist=stats.norm(loc=3000, scale=500),
        user_time_scale='day'
    )
    
    # Complex task DAG (like Airflow example)
    # A >> B >> C >> D
    # B >> E >> D
    # D has two inputs!
    
    A = RiskNode('A', p_fail=0.01, loss_dist=stats.norm(500, 100))
    B = RiskNode('B', p_fail=0.015, loss_dist=stats.norm(600, 120))
    C = RiskNode('C', p_fail=0.05, loss_dist=stats.norm(1000, 200))
    D = RiskNode('D', p_fail=0.02, loss_dist=stats.norm(800, 150))
    E = RiskNode('E', p_fail=0.04, loss_dist=stats.norm(900, 180))
    
    # Contagion between latent risks (BLACK DOTTED lines)
    cloud @ database | 0.7
    cloud @ network | 0.5
    
    # Latent to tasks
    database >> A  # Cascade: DB down → A ALWAYS fails
    database >> B  # Cascade: DB down → B ALWAYS fails  
    network @ D | 0.7  # Contagion: Network issues → 70% chance D fails
    
    # Task dependencies (SOLID DIRECTED arrows)
    A >> B >> C >> D
    B >> E >> D  # D has two paths!

print(f"Created: {dag}")
print(f"Nodes: {len(dag.nodes)} (5 tasks + 3 latent risks)")
print(f"\nTask structure:")
print(f"  Path 1: A → B → C → D")
print(f"  Path 2: B → E → D")
print(f"  (D has two upstream dependencies)")

Created: RiskDAG('complex_etl', nodes=8, latent=3)
Nodes: 8 (5 tasks + 3 latent risks)

Task structure:
  Path 1: A → B → C → D
  Path 2: B → E → D
  (D has two upstream dependencies)


## 3. Visualize with Directed Arrows


In [10]:
widget = visualize_risk_dag(dag)
widget

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'LR', 'nodeSep': 50, 'rankSep': 100, 'animate': …

## 4. Run Monte Carlo Simulation

In [11]:
# Run 1000 simulations
results = dag.run_monte_carlo(n_simulations=5000, seed=42)

print("Simulation complete")

Simulation complete


## 5. Analyze Results

In [12]:
stats_summary = results.summary_statistics()

print("Risk Metrics:")
print(f"  Mean Loss:     ${stats_summary['mean_loss']:,.2f}")
print(f"  95% VaR:       ${stats_summary['var_95']:,.2f}")
print(f"  99% VaR:       ${stats_summary['var_99']:,.2f}")
print(f"  95% ES:        ${stats_summary['es_95']:,.2f}")

print("\nTask Failure Rates:")
for task in ['A', 'B', 'C', 'D', 'E']:
    rate = results.node_failure_rate(task)
    print(f"  {task}: {rate:6.2%}")

print("\nLatent Risk Triggers:")
for latent in ['cloud', 'database', 'network']:
    rate = results.node_failure_rate(latent)
    print(f"  {latent}: {rate:6.2%}")

Risk Metrics:
  Mean Loss:     $289.36
  95% VaR:       $1,979.30
  99% VaR:       $3,748.98
  95% ES:        $3,302.00

Task Failure Rates:
  A:  1.42%
  B:  3.02%
  C:  8.12%
  D: 13.28%
  E:  6.82%

Latent Risk Triggers:
  cloud:  0.02%
  database:  0.14%
  network:  0.04%


## 6. Exceedance Curve

In [13]:
from riskdag.visualization import create_interactive_dashboard

dashboard = create_interactive_dashboard(dag, results, es_percentile=0.95)
widget = dashboard.create_dashboard()
widget